# Installing packages

In [209]:
!pip install elasticsearch
!pip install jproperties

# importing files

In [210]:
from elasticsearch import Elasticsearch
import elasticsearch
from jproperties import Properties
import pickle
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [212]:
config = Properties()

with open('/content/drive/MyDrive/config/elastic_search_config.prop','rb') as f:
  config.load(f)

In [85]:
with open('/content/drive/MyDrive/Pickle files/Product_level_dataframe','rb') as f:
  df = pickle.load(f)

# Loading dataframe

In [86]:
df.reset_index(drop=False,inplace=True)
df

,index,gender,category,sub-category,product-number,product-name,path
0,0,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
1,8,women,dresses,casual_and_day_dresses,62629380,multicolor henley t-shirt dress,women/dresses/casual_and_day_dresses/62629380/...
2,11,women,dresses,casual_and_day_dresses,67938089,black stitch dress,women/dresses/casual_and_day_dresses/67938089/...
3,20,women,dresses,casual_and_day_dresses,67981504,blue signature slim short-sleeve shirtdress,women/dresses/casual_and_day_dresses/67981504/...
4,32,women,dresses,casual_and_day_dresses,70969922,multicolor draped floral-print dress,women/dresses/casual_and_day_dresses/70969922/...
...,...,...,...,...,...,...,...
57257,201826,women,tops,t-shirts,91437497,black fight party t-shirt,women/tops/t-shirts/91437497/91437497_1.jpeg
57258,201830,women,tops,t-shirts,91437517,classic tee black,women/tops/t-shirts/91437517/91437517_1.jpeg
57259,201832,women,tops,t-shirts,91437722,black red heart t-shirt,women/tops/t-shirts/91437722/91437722_0.jpeg
57260,201833,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_0.jpeg


# Elastic Search

In [213]:
es = Elasticsearch([{'host':config.get('endpoint').data,'port':config.get('port').data}])

In [214]:
es.ping()

True

In [163]:
def search(search_phrase,number,field):

  # product-name
  words = search_phrase.split(" ")
  json = []
  size = str(number+20)
  for word in words:
    query = {
        "_source":["product-name","product-number","category","sub-category","image-path","_id"],
        "size":size,
        "query":{
            "match":{
                field: word
            }
        }
    }
    #print(query)
    json.append(es.search(index="product-level", doc_type="_doc",body=query))

  return json

In [192]:
def search_phrases(search_phrase,number,field):

  # product-name
  #words = search_phrase.split(" ")
  #json = []
  size = str(number+20)
  #for word in words:
  query = {
        "_source":["product-name","product-number","category","sub-category","image-path","_id"],
        "size":size,
        "query":{
            "match_phrase":{
                field: search_phrase
            }
        }
  }
    #print(query)
  json=es.search(index="product-level", doc_type="_doc",body=query)

  p_id = []
  score =[]

  if len(json['hits']['hits'])>0:
    p_id = json['hits']['hits'][0]['_source']['product-number']
    score = json['hits']['hits'][0]['_score']
    search_phrase_value = pd.DataFrame(pd.Series({'product-number':p_id,'score':score})).T
    return search_phrase_value
  else:
    return pd.DataFrame({'product-number':p_id,'score':score})
  

In [165]:
def agg_json(json_field):
    
    p_id = []
    score = []

    for file in json_field:
      #print(file)
      for record in file['hits']['hits']:
        p_id.append(record['_source']['product-number'])
        score.append(record['_score'])
    prod_no = pd.DataFrame({'product-number':p_id,"score":score})
    #prod_no.sort_values(by='index_id',inplace=True)
    #prod_no['product-number']=df[df['level_0'].isin(prod_no['index_id'])]['product-number'].values
    prod_no = prod_no.groupby('product-number').agg(sum).reset_index(drop = False).sort_values('score',ascending=False)
    #print(prod_no.info())
    #print(score)
    #print(p_id)
    return prod_no

def call_field(search_phrase, number):

  fields = ['product-name','category','sub-category']
  json_field = []
  for field in fields:
    json_field.append(search(search_phrase,number,field))
  #print(json_field[0][1])
  #print([len(json_field)])
  #print([len(json_field[i]) for i in range(3)])
  #return json_field
  
  prod_no = agg_json(json_field[0])
  #print(prod_no)
  category = agg_json(json_field[1])
  sub_category = agg_json(json_field[2])

  return prod_no, category,sub_category

In [168]:
def call_functions(search_phrase1,number):
  
  prod_no1, category1,sub_category1 = call_field(search_phrase1,number)
  search_phrase_value = search_phrases(search_phrase1,number,'product-name')

  return prod_no1, category1,sub_category1,search_phrase_value

In [193]:
obj =prod_no1, category1,sub_category1,search_phrase_value = call_functions('brown oversized pants',10)

/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in search requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [198]:
bina = []
for x in obj:
  if len(x)>0:
    bina.append(1)
  else:
    bina.append(0)
bina = np.array(bina)

In [159]:
search_phrase('black knit midi dress',10,'product-name')

/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in search requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


,product-number,score
0,56037632,10.7081


In [204]:
def score(prod_no,category,sub_category,search_phrase,a,b,c,d):
  prod_no['score'] = a*prod_no['score']
  category['score'] = b*category['score']
  sub_category['score'] = c*sub_category['score']
  search_phrase['score'] = d*search_phrase['score']

  interim = prod_no.merge(category, how='outer',on='product-number')
  interim1 = interim.merge(sub_category,how='outer',on='product-number')
  interim1.rename(columns={'score_x':'product-name','score_y':'category','score':'sub-category'},inplace=True)
  final = interim1.merge(search_phrase,how='outer',on='product-number',)

  final.fillna(0,inplace=True)
  final['score_final'] = final['product-name']+final['category']+final['sub-category']+final['score']
  final.sort_values(by='score_final', ascending=False,inplace=True)
  final = final.iloc[:20]
  return final

In [205]:
probability = np.array([0.35,0.04,0.01,0.60])
bina = np.array([1,1,0,0])
revised_prob = probability* bina
revised_prob = revised_prob/sum(revised_prob)
revised_prob

array([0.8974359, 0.1025641, 0.       , 0.       ])

In [206]:

frame = score(prod_no1,category1,sub_category1,search_phrase_value,*revised_prob)

In [207]:
frame

,product-number,product-name,category,sub-category,score,score_final
0,90791048,5.668433,0.0,0.0,0.0,5.668433
2,91401942,5.668433,0.0,0.0,0.0,5.668433
3,90779617,5.668433,0.0,0.0,0.0,5.668433
4,89422093,5.668433,0.0,0.0,0.0,5.668433
5,91351721,5.668433,0.0,0.0,0.0,5.668433
6,91339306,5.668433,0.0,0.0,0.0,5.668433
7,88311213,5.668433,0.0,0.0,0.0,5.668433
8,91040275,5.668433,0.0,0.0,0.0,5.668433
1,90358942,5.668433,0.0,0.0,0.0,5.668433
15,90338267,5.570716,0.0,0.0,0.0,5.570716


In [208]:
#df[df['product-number']=='90544677']
#df[df['product-number']=='89050990']
df[df['product-number'].isin(frame.head(20)['product-number'])]

,index,gender,category,sub-category,product-number,product-name,path
449,1492,women,dresses,maxi_and_long_dresses,89227340,blue oversized dress,women/dresses/maxi_and_long_dresses/89227340/8...
2612,9248,women,dresses,casual_and_day_dresses,91339306,brown,women/dresses/casual_and_day_dresses/91339306/...
16695,58118,women,dresses,prom_and_formal_dresses,91040275,brown,women/dresses/prom_and_formal_dresses/91040275...
18830,65439,women,jackets,casual_jackets,89422093,brown,women/jackets/casual_jackets/89422093/89422093...
20684,71455,women,jackets,padded_and_down_jackets,88311213,brown,women/jackets/padded_and_down_jackets/88311213...
34537,119779,women,skirts,mid_length_skirts,90779617,brown,women/skirts/mid_length_skirts/90779617/907796...
36509,126742,women,skirts,knee_length_skirts,91401942,brown,women/skirts/knee_length_skirts/91401942/91401...
38377,133561,women,skirts,mid_length_skirts,90791048,brown,women/skirts/mid_length_skirts/90791048/907910...
38647,134387,women,skirts,mid_length_skirts,91351721,brown,women/skirts/mid_length_skirts/91351721/913517...
41668,146209,women,skirts,mid_length_skirts,90358942,brown,women/skirts/mid_length_skirts/90358942/903589...


In [211]:
with open('/content/drive/MyDrive/Pickle files/final_dataframe','rb')as f:
  image_df = pickle.load(f)

In [217]:
def query_es(user_query):
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, doc['image-vector'])",
                "params": {"query_vector": user_query}
                        },
                         "aggs":{
                             "dimension":{
                                 "terms": {
                                     "field":"product-number"
                                 },
                                 "aggs":{
                                     "Max_score":{
                                         "max":{
                                             "field":"_score"}
                                     }
                                 }
                             }
                         }
                    }
                }
    cards = []

    try:    
      with Elasticsearch([{"host": host, "port":"9200"}]) as es: 
          res = es.search(index='image-level',body={"from":0,"size": 5,"query": script_query})
          for dic in res['hits']['hits']:
              prod_no = dic['_source']['product-number']
              pic_id = dic['_id']-1
              score = dic['_score']
              picture = dic['_source']['image-path']  #url_process_imgdict(df[df["level_0"]==pic_id].to_dict("records")[0]["path"])
              cards.append({"prod_id" : prod_no, "pic_id" : pic_id, "score":score, "img" : picture})
    except Exception as e:
        print(e)   
    return cards

IndentationError: ignored

SyntaxError: ignored